Задание:

Загрузите данные, проверьте правильность, наличие пропущенных значений, типы данных.

Создайте новый признак – марку автомобиля (company). Машины каких производителей встречаются в датасете? Далее исправьте названия и проверьте изменения.

Оставьте только часть предикторов, после чего посчитайте корреляцию между price и другими переменными.

Преобразуйте категориальные переменные с помощью pd.get_dummies(). 

Постройте модель с одним предиктором цены – horsepower. Какой процент изменчивости объясняет полученная модель? (\(R^2\))

Далее – две модели (со всеми предикторами и со всеми, кроме марок машин). Обратите внимание на изменения в \(R^2\), коэффициентах и их значимости. Какую модель лучше оставить? 

Заполните пропуски в результатах.

In [2]:
import pandas as pd
import numpy as np

import scipy.stats as ss
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from statsmodels.stats.api import anova_lm
from statsmodels.stats.multicomp import (pairwise_tukeyhsd,
                                         MultiComparison)
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-m-tulnikov-27/STATISTIKA/cars.csv')

In [4]:
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [5]:
df.dtypes

car_ID                int64
symboling             int64
CarName              object
fueltype             object
aspiration           object
doornumber           object
carbody              object
drivewheel           object
enginelocation       object
wheelbase           float64
carlength           float64
carwidth            float64
carheight           float64
curbweight            int64
enginetype           object
cylindernumber       object
enginesize            int64
fuelsystem           object
boreratio           float64
stroke              float64
compressionratio    float64
horsepower            int64
peakrpm               int64
citympg               int64
highwaympg            int64
price               float64
dtype: object

In [6]:
df.isna().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

Использовать полное название машины – не самый хороший вариант, поэтому создадим новый признак – марку автомобиля (company).

In [7]:
df['company'] = df.CarName.apply(lambda x: x.split(' ')[0])

/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [8]:
df.company.nunique()

28

Часть марок написана с ошибками, исправим

In [9]:
df.company.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'maxda', 'mazda', 'buick', 'mercury',
       'mitsubishi', 'Nissan', 'nissan', 'peugeot', 'plymouth', 'porsche',
       'porcshce', 'renault', 'saab', 'subaru', 'toyota', 'toyouta',
       'vokswagen', 'volkswagen', 'vw', 'volvo'], dtype=object)

In [10]:
df.company = df.company.str.lower().replace({'maxda': 'mazda',
                                             'porcshce': 'porsche',
                                             'toyouta': 'toyota',
                                             'vokswagen': 'volkswagen',
                                             'vw': 'volkswagen'})

In [11]:
df.company.nunique()

22

In [12]:
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,company
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi


Чтобы не перегружать модель большим количеством предикторов, оставим только часть из них:

In [25]:
df_drop = df.drop(columns = {'car_ID', 'symboling', 'CarName', 'doornumber',
                   'enginelocation', 'fuelsystem', 'stroke',
                   'compressionratio', 'peakrpm', 'citympg',
                    'highwaympg'})

In [26]:
df_drop.head()

,fueltype,aspiration,carbody,drivewheel,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,boreratio,horsepower,price,company
0,gas,std,convertible,rwd,88.6,168.8,64.1,48.8,2548,dohc,four,130,3.47,111,13495.0,alfa-romero
1,gas,std,convertible,rwd,88.6,168.8,64.1,48.8,2548,dohc,four,130,3.47,111,16500.0,alfa-romero
2,gas,std,hatchback,rwd,94.5,171.2,65.5,52.4,2823,ohcv,six,152,2.68,154,16500.0,alfa-romero
3,gas,std,sedan,fwd,99.8,176.6,66.2,54.3,2337,ohc,four,109,3.19,102,13950.0,audi
4,gas,std,sedan,4wd,99.4,176.6,66.4,54.3,2824,ohc,five,136,3.19,115,17450.0,audi


После этого посчитаем корреляцию между price и другими переменными.

In [70]:
df_drop.corr(method='pearson').round(2)

,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,horsepower,price
wheelbase,1.00,0.87,0.80,0.59,0.78,0.57,0.49,0.35,0.58
carlength,0.87,1.00,0.84,0.49,0.88,0.68,0.61,0.55,0.68
carwidth,0.80,0.84,1.00,0.28,0.87,0.74,0.56,0.64,0.76
carheight,0.59,0.49,0.28,1.00,0.30,0.07,0.17,-0.11,0.12
curbweight,0.78,0.88,0.87,0.30,1.00,0.85,0.65,0.75,0.84
enginesize,0.57,0.68,0.74,0.07,0.85,1.00,0.58,0.81,0.87
boreratio,0.49,0.61,0.56,0.17,0.65,0.58,1.00,0.57,0.55
horsepower,0.35,0.55,0.64,-0.11,0.75,0.81,0.57,1.00,0.81
price,0.58,0.68,0.76,0.12,0.84,0.87,0.55,0.81,1.00


Последний шаг в подготовке данных: линейная регрессия в python не справляется с категориальными переменными (типом object в pandas), поэтому давайте применим функцию под названием pd.get_dummies().

In [72]:
df_drop.dtypes

fueltype           object
aspiration         object
carbody            object
drivewheel         object
wheelbase         float64
carlength         float64
carwidth          float64
carheight         float64
curbweight          int64
enginetype         object
cylindernumber     object
enginesize          int64
boreratio         float64
horsepower          int64
price             float64
company            object
dtype: object

In [76]:
df_dummy = pd \
.get_dummies(data=df_drop[['company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower', 'price']], drop_first = True)

In [78]:
df_dummy.head()

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price,company_audi,company_bmw,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,88.6,168.8,64.1,2548,130,3.47,111,13495.0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,88.6,168.8,64.1,2548,130,3.47,111,16500.0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,94.5,171.2,65.5,2823,152,2.68,154,16500.0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,99.8,176.6,66.2,2337,109,3.19,102,13950.0,1,0,...,1,0,0,0,0,1,0,0,0,0
4,99.4,176.6,66.4,2824,136,3.19,115,17450.0,1,0,...,1,0,0,0,1,0,0,0,0,0


Сначала построим небольшую модель всего с одним предиктором цены (price) – horsepower.

In [79]:
model = smf.ols(formula="price ~ horsepower", data=df_dummy).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Thu, 29 Dec 2022   Prob (F-statistic):           1.48e-48
Time:                        18:50:11   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.0

R-squared = 65% изменчивости объясняет полученная модель

Теперь построим ещё две модели:

модель со всеми предикторами

модель со всеми предикторами, кроме марок машин

In [81]:
df_dummy.columns

Index(['wheelbase', 'carlength', 'carwidth', 'curbweight', 'enginesize',
       'boreratio', 'horsepower', 'price', 'company_audi', 'company_bmw',
       'company_buick', 'company_chevrolet', 'company_dodge', 'company_honda',
       'company_isuzu', 'company_jaguar', 'company_mazda', 'company_mercury',
       'company_mitsubishi', 'company_nissan', 'company_peugeot',
       'company_plymouth', 'company_porsche', 'company_renault',
       'company_saab', 'company_subaru', 'company_toyota',
       'company_volkswagen', 'company_volvo', 'fueltype_gas',
       'aspiration_turbo', 'carbody_hardtop', 'carbody_hatchback',
       'carbody_sedan', 'carbody_wagon', 'drivewheel_fwd', 'drivewheel_rwd',
       'enginetype_dohcv', 'enginetype_l', 'enginetype_ohc', 'enginetype_ohcf',
       'enginetype_ohcv', 'enginetype_rotor', 'cylindernumber_five',
       'cylindernumber_four', 'cylindernumber_six', 'cylindernumber_three',
       'cylindernumber_twelve', 'cylindernumber_two'],
      dtype='object'

модель со всеми предикторами

In [90]:
X = df_dummy.iloc[:, 1:-1]
Y = df_dummy.iloc[:, -42]
X = sm.add_constant(X)
model_full = sm.OLS(Y, X)
model_full.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 5.616e+29
Date:                Thu, 29 Dec 2022   Prob (F-statistic):               0.00
Time:                        19:04:44   Log-Likelihood:                 4762.4
No. Observations:                 205   AIC:                            -9431.
Df Residuals:                     158   BIC:                            -9275.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  2.731e-12   1.47e-10      0.019      0.985   -2.88e-10    2.94e-10
carlength              1.798e-13   5.41e-13      0.333      0.740   -8.88e-13    1.25e-12
carwidth              -1.174e-12   2.72e-12     -0.432      0.666   -6.54e-12    4.19e-12
curbweight             5.568e-15   1.97e-14      0.282      0.778   -3.34e-14    4.46e-14
enginesize            -1.185e-13   2.95e-13     -0.401      0.689   -7.02e-13    4.65e-13
boreratio               3.81e-12    2.2e-11      0.173      0.863   -3.96e-11    4.73e-11
horsepower            -3.031e-14   2.22e-13     -0.136      0.892   -4.69e-13    4.08e-13
price                     1.0000   9.65e-16   1.04e+15      0.000       1.000       1.000
company_audi           1.045e-11   2.59e-11      0.403      0.687   -4.07e-11    6.16e-11
company_bmw            1.387e-11   2.68e-11      0.518      0.605    -3.9e-11    6.68e-11
company_buick          9.013e-12   2.97e-11      0.304      0.762   -4.96e-11    6.76e-11
company_chevrolet      2.162e-11   2.49e-11      0.870      0.386   -2.75e-11    7.07e-11
company_dodge          1.103e-11   2.05e-11      0.539      0.591   -2.94e-11    5.15e-11
company_honda          1.155e-11   1.98e-11      0.585      0.560   -2.75e-11    5.06e-11
company_isuzu          4.257e-12   2.21e-11      0.193      0.847   -3.93e-11    4.78e-11
company_jaguar         1.959e-11   3.07e-11      0.639      0.524    -4.1e-11    8.02e-11
company_mazda          1.195e-11   2.01e-11      0.594      0.554   -2.78e-11    5.17e-11
company_mercury        1.124e-11   3.35e-11      0.335      0.738    -5.5e-11    7.75e-11
company_mitsubishi      1.85e-11   2.04e-11      0.906      0.366   -2.18e-11    5.88e-11
company_nissan         1.003e-11   1.92e-11      0.524      0.601   -2.78e-11    4.79e-11
company_peugeot        1.823e-11   2.15e-11      0.849      0.397   -2.42e-11    6.06e-11
company_plymouth       1.604e-11   2.04e-11      0.786      0.433   -2.43e-11    5.63e-11
company_porsche        6.233e-12   3.47e-11      0.180      0.857   -6.22e-11    7.47e-11
company_renault        7.723e-12   2.56e-11      0.302      0.763   -4.28e-11    5.82e-11
company_saab           8.928e-12   2.17e-11      0.412      0.681   -3.39e-11    5.18e-11
company_subaru        -1.096e-11    4.3e-11     -0.255      0.799   -9.58e-11    7.39e-11
company_toyota         5.512e-12   1.86e-11      0.296      0.768   -3.13e-11    4.23e-11
company_volkswagen     1.536e-11   1.96e-11      0.783      0.435   -2.34e-11    5.41e-11
company_volvo          2.519e-11    2.3e-11      1.097      0.274   -2.02e-11    7.06e-11
fueltype_gas          -1.908e-12   1.11e-11     -0.171      0.864   -2.39e-11    2.01e-11
aspiration_turbo      -1.932e-12   8.32e-12     -0.232      0.817   -1.84e-11    1.45e-11
carbody_hardtop        

модель со всеми предикторами, кроме марок машин

In [96]:
df_drop_company = df_dummy.columns[~df_dummy.columns.str.startswith('company_')]

In [104]:
X_2 = df_dummy[df_drop_company].drop('price', axis = 'columns')
Y_2 = df_dummy['price']
X_2 = sm.add_constant(X_2)
model_drop_company = sm.OLS(Y_2, X_2)
model_drop_company.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                Thu, 29 Dec 2022   Prob (F-statistic):           9.86e-81
Time:                        19:19:48   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -1.7e+04    1.3e+04     -1.309      0.192   -4.26e+04    8625.219
wheelbase                71.1868     87.028      0.818      0.414    -100.552     242.925
carlength               -51.3497     50.341     -1.020      0.309    -150.692      47.993
carwidth                541.8700    253.327      2.139      0.034      41.958    1041.782
curbweight                2.9577      1.796      1.647      0.101      -0.585       6.501
enginesize               36.0515     22.376      1.611      0.109      -8.105      80.208
boreratio             -2230.4519   1731.681     -1.288      0.199   -5647.719    1186.815
horsepower               86.8164     16.717      5.193      0.000      53.827     119.806
fueltype_gas          -2423.0935    975.579     -2.484      0.014   -4348.283    -497.904
aspiration_turbo      -1139.2334    787.230     -1.447      0.150   -2692.738     414.271
carbody_hardtop       -3728.0949   1519.366     -2.454      0.015   -6726.383    -729.807
carbody_hatchback     -4639.1279   1294.594     -3.583      0.000   -7193.855   -2084.401
carbody_sedan         -3337.7854   1356.075     -2.461      0.015   -6013.838    -661.732
carbody_wagon         -4315.4661   1465.089     -2.946      0.004   -7206.646   -1424.287
drivewheel_fwd          297.2402   1121.766      0.265      0.791   -1916.431    2510.912
drivewheel_rwd         1939.1594   1226.536      1.581      0.116    -481.263    4359.582
enginetype_dohcv      -1.713e+04   4540.120     -3.773      0.000   -2.61e+04   -8171.142
enginetype_l           2176.0521   1332.818      1.633      0.104    -454.106    4806.211
enginetype_ohc         4006.5687    897.437      4.464      0.000    2235.584    5777.553
enginetype_ohcf        5421.2822   1388.078      3.906      0.000    2682.076    8160.488
enginetype_ohcv       -5001.2242   1345.435     -3.717      0.000   -7656.280   -2346.169
enginetype_rotor      -5196.6413   2084.532     -2.493      0.014   -9310.217   -1083.066
cylindernumber_five   -1.389e+04   2891.599     -4.802      0.000   -1.96e+04   -8180.353
cylindernumber_four   -1.618e+04   3243.610     -4.989      0.000   -2.26e+04   -9780.800
cylindernumber_six    -1.175e+04   2287.516     -5.135      0.000   -1.63e+04   -7231.547
cylindernumber_three  -9264.0553   4750.899     -1.950      0.053   -1.86e+04     111.279
cylindernumber_twelve -1.214e+04   3386.507     -3.585      0.000   -1.88e+04   -5458.378
cylindernumber_two    -5196.6413   2084.532     -2.493      0.014   -9310.217   -1083.066
==============================================================================
Omnibus:                       18.493   Durbin-Watson:                   1.249
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.728
Skew:                           0.293   Prob(JB):       

Если судить чисто по диагностическим показателям (вроде R2), то модель со всеми предикторами лучшая

Большинство коэффициентов, связанных с марками машин, статистически незначимы

что хотя марки машин и объясняют какую-то часть общей дисперсии в данных, эта часть не самая большая - около 4%. На фоне того, как эта переменная усложняет модель дополнительными статнезначимыми коэффициентами, мы можем принять решение выкинуть её из модели либо дополнительно переделать. Однозначно правильного решения тут нет.

Но допустим, что мы действительно решили избавиться от этого предиктора и взять вторую модель с предыдущего шага, то выбранная модель объясняет примерно 
90
 % дисперсии. Среди предикторов 
10
 из 27 оказались не значимыми (p > 0.05). Пример интерпретации: при единичном изменении показателя horsepower, цена 
возрастает
 на 
86.8164.